In [4]:
import subprocess
from pathlib import Path

# List of mapping files to test
mapping_files = [
    "../data/bfo-mapping.ttl",
    "../data/ies-mapping.ttl",
    "../data/ccom-mapping.ttl",
    "../data/qudt-mapping.ttl",
    "../data/ccot-mapping.ttl",
    "../data/to-mapping.ttl"
]

# Reasoners to run (ROBOT supports HermiT and ELK)
reasoners = ["HermiT", "ELK"]

# Create an output directory for classified ontologies
outdir = Path("classified/")
outdir.mkdir(exist_ok=True)

def run_reasoner(input_file, reasoner):
    output_file = outdir / f"{Path(input_file).stem}-{reasoner}-classified.ttl"
    print(f"[RUN] {reasoner} on {input_file} -> {output_file}")
    
    result = subprocess.run([
        "robot", "reason",
        "--reasoner", reasoner,
        "--input", input_file,
        "--output", str(output_file)
    ], capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"[OK] {reasoner} finished successfully")
    else:
        print(f"[FAIL] {reasoner} on {input_file}")
        print(result.stderr)

# Loop over all files and reasoners
for f in mapping_files:
    for r in reasoners:
        run_reasoner(f, r)


[RUN] HermiT on ../data/bfo-mapping.ttl -> classified/bfo-mapping-HermiT-classified.ttl
[OK] HermiT finished successfully
[RUN] ELK on ../data/bfo-mapping.ttl -> classified/bfo-mapping-ELK-classified.ttl
[OK] ELK finished successfully
[RUN] HermiT on ../data/ies-mapping.ttl -> classified/ies-mapping-HermiT-classified.ttl
[OK] HermiT finished successfully
[RUN] ELK on ../data/ies-mapping.ttl -> classified/ies-mapping-ELK-classified.ttl
[OK] ELK finished successfully
[RUN] HermiT on ../data/ccom-mapping.ttl -> classified/ccom-mapping-HermiT-classified.ttl
[OK] HermiT finished successfully
[RUN] ELK on ../data/ccom-mapping.ttl -> classified/ccom-mapping-ELK-classified.ttl
[OK] ELK finished successfully
[RUN] HermiT on ../data/qudt-mapping.ttl -> classified/qudt-mapping-HermiT-classified.ttl
[OK] HermiT finished successfully
[RUN] ELK on ../data/qudt-mapping.ttl -> classified/qudt-mapping-ELK-classified.ttl
[OK] ELK finished successfully
[RUN] HermiT on ../data/ccot-mapping.ttl -> classifi